In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Data Preparation
df = pd.read_csv('DailyDelhiClimate.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)  # Set 'date' as index

# Features to use
features = ['meantemp', 'humidity', 'wind_speed', 'meanpressure']
df = df[features]

# Scaling
scalers = {}
for column in df.columns:
    scaler = MinMaxScaler()
    df[column] = scaler.fit_transform(df[[column]])
    scalers['scaler_' + column] = scaler

# Define n_past, n_future, and n_features
n_past = 10  # Number of past time steps to use
n_future = 5  # Number of future time steps to predict
n_features = len(features)

# Create sequences (sliding window approach)
X = []
y = []
for i in range(n_past, len(df) - n_future + 1):
    X.append(df.iloc[i - n_past:i].values)
    y.append(df.iloc[i:i + n_future].values)

X = np.array(X)
y = np.array(y)

# Train/Test Split (Temporal split - 80:20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 2. VAE BiLSTM Encoder-Decoder Model

# Encoder
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(encoder_inputs)
encoder_bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(encoder_bilstm)

# Variational layers
z_mean = tf.keras.layers.Dense(16, name="z_mean")(encoder_bilstm)
z_log_var = tf.keras.layers.Dense(16, name="z_log_var")(encoder_bilstm)

# Define a custom layer for KL divergence calculation
class KLDivergenceLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(KLDivergenceLayer, self).__init__(**kwargs)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        # Add the KL loss to the model's losses
        self.add_loss(kl_loss)
        return z_mean  # You can return z_mean or any other relevant output

# Apply the custom layer
z_mean = KLDivergenceLayer()([z_mean, z_log_var])

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = tf.keras.layers.Lambda(sampling, name="z")([z_mean, z_log_var])

# Decoder
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(z)
decoder_bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(decoder_inputs)
decoder_bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(decoder_bilstm)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_bilstm)

# VAE model
vae = tf.keras.models.Model(encoder_inputs, decoder_outputs, name="vae_bilstm")

# 3. Compilation and Training
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

vae.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_vae = vae.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=16, verbose=1, callbacks=[reduce_lr, early_stopping])

# 4. Prediction and Inverse Scaling
pred_vae = vae.predict(X_test)

for index, i in enumerate(features):  # Iterate through the feature names
    scaler = scalers['scaler_' + i]
    pred_vae[:, :, index] = scaler.inverse_transform(pred_vae[:, :, index])
    y_train[:, :, index] = scaler.inverse_transform(y_train[:, :, index])
    y_test[:, :, index] = scaler.inverse_transform(y_test[:, :, index])

# 5. Evaluation (for the whole model across all 5 predicted days)

# Define function for Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Flatten the true and predicted values for overall comparison
y_true_all = y_test.flatten()
y_pred_vae_all = pred_vae.flatten()

# Compute metrics for VAE model
mae_vae = mean_absolute_error(y_true_all, y_pred_vae_all)
mse_vae = mean_squared_error(y_true_all, y_pred_vae_all)
rmse_vae = np.sqrt(mse_vae)
mape_vae = mean_absolute_percentage_error(y_true_all, y_pred_vae_all)
r2_vae = r2_score(y_true_all, y_pred_vae_all)

# Display results in a structured table
results_df = pd.DataFrame({
    "Model": ["VAE BiLSTM"],
    "MAE": [mae_vae],
    "MSE": [mse_vae],
    "RMSE": [rmse_vae],
    "MAPE": [mape_vae],
    "R2 Score": [r2_vae]
})

print(results_df)

# Reshape y_test and pred_vae to 2D
y_test_2d = y_test.reshape(-1, y_test.shape[-1])
pred_vae_2d = pred_vae.reshape(-1, pred_vae.shape[-1])

# Normalize MSE using variance of actual data
y_var = np.var(y_test_2d)
normalized_mse_vae = mean_squared_error(y_test_2d, pred_vae_2d) / y_var

print("Normalized MSE VAE:", normalized_mse_vae)

# Predict next 10 days
X_test_last = X_test[-1:].reshape(1, n_past, n_features)
pred_next_10_days_vae = vae.predict(X_test_last)

# Inverse transform the predicted values
for index, i in enumerate(features):
    scaler = scalers['scaler_' + i]
    pred_next_10_days_vae[:, :, index] = scaler.inverse_transform(pred_next_10_days_vae[:, :, index])

print("Predicted next 10 days (VAE):", pred_next_10_days_vae)